## This Notebook demos how to run unbiased area estimation

In [ ]:
%pip install -e .

In [ ]:
# Import all the required functions

from unbiased_area_estimation.utils import read_config
from unbiased_area_estimation.unbiased_estimation import AreaEstimator
from unbiased_area_estimation.sampling_allocation import create_sample_allocator

In [ ]:
# Load your config file
CONFIG_PATH = 'config.json' # Set the path to your config file here
config = read_config(CONFIG_PATH)

In [ ]:
#preprocess the raster and masks

sample_allocator = create_sample_allocator(config['sampling'])

area_estimator = AreaEstimator(
    raster_path=config['raster_path'],
    mask_paths=config['mask_paths'],
    class_merge_dict=config['class_merge_dict'],
    epsg=config['epsg'],
    sample_allocator=sample_allocator,
    temp_dir=config['output_dir'] if config['keep_preprocessed_files'] else None,
    output_dir=config['output_dir'],
    overwrite_existing=False,
    )

area_estimator.preprocess_files()

In [ ]:
# Create the initial sample designs
sampling_designs = area_estimator.create_sample_designs()
print(sampling_designs)

In [ ]:
# Adapt sampling designs

#sampling_designs = {} # Copy the output from the cell above and modify according to your requirements.
target_errors = area_estimator.get_expected_target_errors(sampling_designs)
print(target_errors)

In [ ]:
# Create the actual sample sets by sampling the raster with the number of samples per class
sample_sets = area_estimator.create_sample_sets(sampling_designs)
area_estimator.save_sample_sets(sample_sets=sample_sets, single_file=True)

In [ ]:
# You should now annotate your samples. Once you have done that, you can run the following code to estimate the area of each class in the raster.
# Use the original csv to save the annotated samples, if you do not want to change the paths.
from unbiased_area_estimation.utils import compute_unbiased_areas_from_ceo
from IPython.display import display, HTML

# Load the annotated samples
ANNOTATED_SAMPLES_FOLDER = config['output_dir']
annotated_column_name = 'Crop Type' # Change this to the column name in your annotated samples that contains the class labels

unbiased_area_estimates = compute_unbiased_areas_from_ceo(ANNOTATED_SAMPLES_FOLDER, annotated_column_name)

display(HTML(unbiased_area_estimates.to_html()))